# Generate the RepCode circuit

In [1]:
from result_saver import SaverProvider

provider = SaverProvider()

In [5]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import get_repcode_layout

DISTANCE = 3
ROUNDS = 3
DEVICE = "ibmq_mumbai"
LOGICAL = '1'
SHOTS = 1e4

code = RepetitionCodeCircuit(DISTANCE, ROUNDS)
backend = provider.get_backend(DEVICE)
layout = get_repcode_layout(DISTANCE, backend, _is_hex=False)

qc = code.circuit[LOGICAL]                                   

Finding the longest path starting from 27 qubits: 100%|██████████| 27/27 [00:00<00:00, 24726.25it/s]


# Noise model

In [3]:
from qiskit_qec.noise import PauliNoiseModel

def get_noise_model(p1Q, p2Q, pXY, pZ, pRO, pRE):

    error_dict = {
        'reset': {
            "chan": {
                        'i':1-pRE,
                        'x':pRE
                    }
                },
        'measure': {
            "chan": {
                        'i':1-pRO,
                        'x':pRO
                    }
                },
        'h': {
            "chan": {
                        'i':1-p1Q
                    }|
                    {
                        i:p1Q/3
                        for i in 'xyz'
                    }
                },
        #idling error attached to a custom gate "idle_1" (assumed to be the identity acting on a single qubit)
        'idle_1': {
            "chan": {
                        'i':1-pXY,
                        'x':pXY/2,
                        'y':pXY/2
                    }
                },
        #another type of idling error attached to a custom gate "idle_2" (assumed to be the identity acting on a single qubit)
        'idle_2': {
            "chan": {
                        'i':1-pZ,
                        'z':pZ
                    }
                },

        'cx': {
            "chan": {
                        'ii':1-p2Q
                    }|
                    {
                        i+j:p2Q/15
                        for i in 'ixyz'
                        for j in 'ixyz'
                        if i+j!='ii'
                    }
                },
        'swap': {
            "chan": {
                        'ii':1-p2Q
                    }|
                    {
                        i+j:p2Q/15
                        for i in 'ixyz'
                        for j in 'ixyz'
                        if i+j!='ii'
                    }
                }
                }

    noise_model = PauliNoiseModel(fromdict=error_dict)

    return noise_model



# Get counts via stim

In [7]:
from qiskit_qec.utils import get_counts_via_stim

noise_model = get_noise_model(p1Q=1e-4, p2Q=6e-3, pXY=1e-4, pZ=1e-4, pRO=1e-1, pRE=0)
counts = get_counts_via_stim(qc, shots=int(SHOTS), noise_model=noise_model)
sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
(sorted_counts)

[('111 00 00 00', 3781),
 ('111 10 10 10', 426),
 ('101 00 00 00', 424),
 ('111 10 00 00', 422),
 ('111 01 00 00', 406),
 ('110 00 00 00', 405),
 ('111 01 01 00', 404),
 ('011 00 00 00', 400),
 ('111 01 01 01', 395),
 ('111 10 10 00', 392),
 ('101 01 00 00', 80),
 ('010 00 00 00', 65),
 ('110 01 00 00', 60),
 ('011 10 00 00', 56),
 ('111 11 01 01', 53),
 ('101 10 00 00', 52),
 ('111 11 11 01', 52),
 ('111 00 00 01', 52),
 ('110 10 00 00', 52),
 ('101 10 10 10', 52),
 ('111 00 00 10', 51),
 ('011 10 10 00', 50),
 ('110 10 10 10', 50),
 ('110 10 10 00', 50),
 ('111 00 01 01', 49),
 ('110 01 01 01', 49),
 ('111 11 11 11', 49),
 ('111 00 01 00', 46),
 ('011 01 01 01', 46),
 ('110 01 01 00', 46),
 ('011 01 01 00', 45),
 ('101 01 01 01', 45),
 ('101 01 01 00', 45),
 ('111 11 00 00', 44),
 ('111 00 10 10', 44),
 ('001 00 00 00', 43),
 ('111 11 11 00', 43),
 ('111 11 11 10', 42),
 ('111 11 01 00', 41),
 ('111 11 10 10', 40),
 ('100 00 00 00', 39),
 ('011 10 10 10', 38),
 ('011 01 00 00', 38),


# counts to IQ

In [8]:
from soft_info import get_repcode_IQ_map, get_KDEs

IQ_map = get_repcode_IQ_map(layout, ROUNDS)
print(IQ_map)

mumbai_calib_job = "cne879d72scg008df4j0" # 21.11.23
kde_dict, scaler_dict = get_KDEs(provider, mumbai_calib_job)

{0: 1, 1: 3, 2: 1, 3: 3, 4: 1, 5: 3, 6: 0, 7: 2, 8: 5}
Specified job execution date: 2023-11-21 19:54:05.610765+01:00
Found jobs for backend ibmq_mumbai with closest execution date 2023-11-21 18:54:05.610765+00:00.


In [9]:
# sample from the kde

kde0, kde1 = kde_dict[10]
scaler = scaler_dict[10]
sample = kde0.sample()


# invert the scaling
sample = scaler.inverse_transform(sample)
print(sample)


IQ_point = complex(sample[0][0], sample[0][1])


print(IQ_point)


[[-20827979.42945149 -89806035.96672024]]
(-20827979.429451488-89806035.96672024j)


In [10]:
import numpy as np
from tqdm import tqdm

# Determine the total number of shots and number of qubits
total_shots = sum(shots for _, shots in sorted_counts)
len_IQ_array = len(IQ_map)

# Preallocate the NumPy array
IQ_memory = np.zeros((total_shots, len_IQ_array), dtype=np.complex128)

# Fill the array
shot_idx = 0
for count_str, shots in tqdm(sorted_counts):
    for _ in range(shots):
        num_spaces = 0
        for IQ_idx, bit in enumerate(count_str[:]):  # invert the order of count str to get IQ order
            if bit == ' ':
                num_spaces += 1
                continue
            qubit_idx = IQ_map[IQ_idx - num_spaces]
            [kde0, kde1], scaler = kde_dict[qubit_idx], scaler_dict[qubit_idx]
            if bit == '0':
                sample = scaler.inverse_transform(kde0.sample(1))
                IQ_memory[shot_idx, qubit_idx] = complex(sample[0][0], sample[0][1])
            elif bit == '1':
                sample = scaler.inverse_transform(kde1.sample(1))
                IQ_memory[shot_idx, qubit_idx] = complex(sample[0][0], sample[0][1])
        shot_idx += 1

100%|██████████| 254/254 [00:02<00:00, 85.74it/s] 


In [11]:
import numpy as np
print(np.array(IQ_memory).shape)

(10000, 9)


# Decode

In [29]:
from Scratch import create_or_load_kde_grid

grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, mumbai_calib_job, 300, 2, other_date=None)

Specified job execution date: 2023-11-21 19:54:05.610765+01:00
Found jobs for backend ibmq_mumbai with closest execution date 2023-11-21 18:54:05.610765+00:00.
Searching for ibmq_mumbai and 23.11.21_10h17_300pts_2std


In [13]:
from datetime import datetime

def get_avgs_from_dict(noise_dict):
    '''
    Takes noise dictionary as an argument then returns length 5
    array defining average noise levels: [init, idle, RO, single-, two-qubit gate]
    '''
    init_avg = np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    idle_avg = np.average([noise_dict[qubit]['idle'] for qubit in noise_dict])
    RO_avg = np.average([noise_dict[qubit]['RO'] for qubit in noise_dict])
    single_avg = 1e-4#np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    two_gate_avg = np.average([noise_dict[qubit]['2-gate'][connection]
                        for qubit in noise_dict
                        for connection in noise_dict[qubit]['2-gate']
                        if connection != 'default'])
    return [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]
# End get_avgs_from_dict


def get_noise_dict_from_backend(backend, layout, date=None):
    '''
    Takes a given backend and a mapping object and returns a noise dictionary
    to pass to a new Cross_Platform_Code/Hex_Code to have accurate backend
    specific noise information based on calibration data.
    '''
    # Pre-reqs
    noise_dict = {}
    round_time = 1000e-9
    if date is None:
        t = None
    else:
        t = datetime(day=int(date[-2:]), month=int(date[-4:-2]), year=int(date[:4]))
    properties = backend.properties(datetime=t)
    # Initializing each qubit with single qubit noise
    for qubit in layout:
        # Defining ROI error
        ROI_error = properties.readout_error(qubit)
        # Defining idle error
        t1 = properties.t1(qubit)
        t2 = properties.t2(qubit)
        t_time = min(t1, t2)
        idle_error = 1 - np.exp(-round_time / t_time)
        # Updating dictionary
        noise_dict[qubit] = {}
        noise_dict[qubit]['init'] = ROI_error
        noise_dict[qubit]['idle'] = idle_error
        noise_dict[qubit]['RO'] = ROI_error
        noise_dict[qubit]['gate'] = 1e-4
        noise_dict[qubit]['2-gate'] = {'default': 1e-2}
    # Two qubit gate noise
    all_qiskit_indexes = layout 
    for pair in backend.coupling_map:
        # Making sure connection is in circuit
        if pair[0] in all_qiskit_indexes and pair[1] in all_qiskit_indexes:
            # Getting two qubit gate error
            two_gate_error = properties.gate_error('cx', pair)
            if two_gate_error > .5:
                two_gate_error = .5
            # # Updating dictionary
            # stim_index_0 = map.get_stim_index(qiskit_index=pair[0])
            # stim_index_1 = map.get_stim_index(qiskit_index=pair[1])
            noise_dict[pair[0]]['2-gate'][pair[1]] = two_gate_error
            noise_dict[pair[1]]['2-gate'][pair[0]] = two_gate_error
    return noise_dict
# End get_noise_dict_from_backend

In [30]:

date = "2023-10-30 09:45:17.340386+01:00" #md.creation_date[0] # md[0] for log 1
noise_dict = get_noise_dict_from_backend(provider.get_backend(DEVICE), layout, date=None) # takes noise closest to today
print("noise_dict:", noise_dict)
avgs_noise = get_avgs_from_dict(noise_dict)
print("avgs_noise [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]:", avgs_noise)

# takes 4s

noise_dict: {1: {'init': 0.030000000000000027, 'idle': 0.007897557855443993, 'RO': 0.030000000000000027, 'gate': 0.0001, '2-gate': {'default': 0.01, 0: 0.005455353871420404, 2: 0.010170945774381879}}, 3: {'init': 0.021100000000000008, 'idle': 0.008996902186028333, 'RO': 0.021100000000000008, 'gate': 0.0001, '2-gate': {'default': 0.01, 2: 0.008544514649166696, 5: 0.021703370620595658}}, 0: {'init': 0.47, 'idle': 0.0121859535428267, 'RO': 0.47, 'gate': 0.0001, '2-gate': {'default': 0.01, 1: 0.005455353871420404}}, 2: {'init': 0.012699999999999934, 'idle': 0.008960615196325294, 'RO': 0.012699999999999934, 'gate': 0.0001, '2-gate': {'default': 0.01, 1: 0.010170945774381879, 3: 0.008544514649166696}}, 5: {'init': 0.01870000000000005, 'idle': 0.022334153450303096, 'RO': 0.01870000000000005, 'gate': 0.0001, '2-gate': {'default': 0.01, 3: 0.021703370620595658}}}
avgs_noise [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]: [0.1105, 0.012075036446185483, 0.1105, 0.0001, 0.01146854622889115

In [31]:
import pymatching
import stim


circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization= avgs_noise[4], #two-qubit-fidelity,
                                after_reset_flip_probability= 0, #reset error,
                                before_measure_flip_probability= avgs_noise[2], #measurement error,
                                before_round_data_depolarization= avgs_noise[1]) #idle error)
# print(circuit)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

In [32]:
import cpp_soft_info

matching = pymatching.Matching.from_detector_error_model(model)
# cpp_soft_info.reweight_edges_based_on_error_probs(matching._matching_graph, counts, False, "spitz")

p_data = 6.869e-3 # mean sherbrooke noise
num_errors = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_memory,
                                           ROUNDS, 0, IQ_map, grid_dict,
                                           processed_scaler_dict, p_data=p_data, p_mixed=p_data/1000, #p_mixed=1e-80, for d=30
                                           common_measure=-1, _bimodal=False, merge_strategy = "replace")
print("num_errors:", num_errors, "out of", len(IQ_memory), "shots")

num_errors: 274 out of 10000 shots


In [33]:
matching = pymatching.Matching.from_detector_error_model(model)
num_errors = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, IQ_memory,
                                           ROUNDS, 0, IQ_map, grid_dict,
                                           processed_scaler_dict)
print("num_errors:", num_errors, "out of", len(IQ_memory), "shots")


num_errors: 1564 out of 10000 shots


In [35]:
matching = pymatching.Matching.from_detector_error_model(model)

def weight_to_prob(weight):
    return 1/(1+np.exp(weight))

p_data = 6.869e-3 # mean sherbrooke ECR error
p_mixed = p_data/1 # Same as weighted
# p_meas = 1e-3
p_meas = 15.900e-2 # random found number


# p_data = -1
# p_mixed = -1
# p_meas = -1

num_errors = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, IQ_memory, 
                                           ROUNDS, 0, IQ_map, grid_dict, processed_scaler_dict,
                                           p_data, p_mixed, p_meas, common_measure=-1)

print("num_errors:", num_errors, "out of", len(IQ_memory), "shots")
         
# takes 1s

num_errors: 1843 out of 10000 shots
